In [1]:
import requests
import pandas as pd
import sqlite3 as sq
from time import sleep
import networkx as nx
import itertools
from tqdm import tqdm
import dtale

In [2]:
sq.sqlite_version

'3.39.3'

In [6]:
headers = {'User-Agent': "brizileo@gmail.com"}

### Create list of companies

Documentation of SEC EDGAR https://www.sec.gov/edgar/sec-api-documentation

In [7]:
tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)

In [8]:
companies = []

In [9]:
for c in list(tickers_cik.json().values()):
  companies.append([str(c['cik_str']).zfill(10),c['ticker'],c['title']])

In [10]:
companies[:1]

[['0000320193', 'AAPL', 'Apple Inc.']]

In [29]:
len(companies)

11152

In [30]:
dtale.show(pd.DataFrame(data=companies, columns=['cik','ticker','name']))

### Create SQLite db

In [ ]:
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute('CREATE TABLE tblFact (ticker TEXT,company TEXT,fact TEXT,form TEXT,end TEXT,start TEXT,val NUMERIC,fy TEXT,fp TEXT,filed TEXT,frame TEXT)')
#cursor.execute('DROP TABLE tblFact')
conn.commit()
conn.close()

### Retrieve company data

In [ ]:
#companyFacts = []
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()

#loop through companies and extract facts
for c in companies:
  url = "https://data.sec.gov/api/xbrl/companyfacts/CIK"+c[0]+".json"

  try:
    json=requests.get(url, headers=headers).json()
    sleep(0.12)
    output = []

    #limit the extraction to companies with US-GAAP or IFR-Full accounting standards report
    if 'us-gaap' in json['facts']:
      for f in json['facts']['us-gaap']:
        #limit extraction to companies where facts key has USD units
        if 'USD' in json['facts']['us-gaap'][f]['units']:
          for r in json['facts']['us-gaap'][f]['units']['USD']:
            #limit extrction to form 10-K
            if r['form'] == '10-K':
              if 'frame' in r:
                frame = r['frame']
              else:
                frame = ''
              if 'start' in r:
                start = r['start']
              else:
                start = ''                
              output.append([c[1],c[2],f,r['form'],r['end'],start,r['val'],r['fy'],r['fp'],r['filed'],frame])
              cursor.execute('INSERT INTO tblFact VALUES (?,?,?,?,?,?,?,?,?,?,?)',(c[1],c[2],f,r['form'],r['end'],start,r['val'],r['fy'],r['fp'],r['filed'],frame))              
    elif 'ifrs-full' in json['facts']:
      for f in json['facts']['ifrs-full']:
        #limit extraction to companies where facts key has USD units
        if 'USD' in json['facts']['ifrs-full'][f]['units']:
          for r in json['facts']['ifrs-full'][f]['units']['USD']:
            #limit extrction to form 20-K
            if r['form'] == '20-F':
              if 'frame' in r:
                frame = r['frame']
              else:
                frame = '' 
              if 'start' in r:
                start = r['start']
              else:
                start = ''                                
              output.append([c[1],c[2],f,r['form'],r['end'],start,r['val'],r['fy'],r['fp'],r['filed'],frame])             
              cursor.execute('INSERT INTO tblFact VALUES (?,?,?,?,?,?,?,?,?,?,?)',(c[1],c[2],f,r['form'],r['end'],start,r['val'],r['fy'],r['fp'],r['filed'],frame))     
    else: 
      print('no us-gaap nor ifrs-full '+c[1]+' '+c[0])

    if len(output) == 0:
      print('no USD or Form available '+c[1]+' '+c[0])

  except:
    print('failed '+c[1]+' '+c[0])  
    pass         

conn.commit()
conn.close()

In [ ]:
#Create Master Fact table removing duplicates
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE TABLE  tblMasterFact AS
SELECT *
FROM 
(

    SELECT *, ROW_NUMBER() OVER (PARTITION BY ticker,fact,fy ORDER BY end DESC, julianday(end)-julianday(start) DESC, filed DESC) RN
    FROM tblFact
) X
WHERE RN = 1
"""    
)
conn.commit()
conn.close()
#check tests section to see why ROW_NUMBER is justified

In [ ]:
#drop table tblFact to save space
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute('DROP TABLE tblFact')
conn.commit()
conn.close()

In [ ]:
#Remove companies with null fy
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
DELETE 
FROM tblMasterFact
WHERE ticker IN (
  SELECT DISTINCT ticker
  FROM tblMasterFact
  WHERE fy IS NULL
  )
"""
)
conn.commit()
conn.close()

In [4]:
#Create nodeId
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
ALTER TABLE tblMasterFact ADD nodeID VARCHAR
"""
)
cursor.execute(
"""
UPDATE tblMasterFact
SET nodeID = ticker || '_' || fy
"""
)
conn.commit()
conn.close()

In [45]:
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()

In [17]:
cursor.execute(
"""
ALTER TABLE tblMasterFact ADD percentileVal REAL
"""
)
conn.commit()

In [17]:
#Create normalised fact value based on percentile rank
cursor.execute(
"""
UPDATE tblMasterFact
SET percentileVal = u.pVal
FROM (SELECT *,PERCENT_RANK() OVER (PARTITION BY fact ORDER BY val ASC) AS pVal FROM tblMasterFact) u
WHERE tblMasterFact.nodeId = u.nodeID AND tblMasterFact.fact = u.fact
"""
)

In [46]:
cursor.execute(
"""
ALTER TABLE tblMasterFact ADD numberOfFactsPerNode REAL
"""
)
conn.commit()

In [47]:
#Add column for number of facts per nod
cursor.execute(
"""
UPDATE tblMasterFact
SET numberOfFactsPerNode = u.nFacts
FROM (SELECT *,COUNT(*) OVER (PARTITION BY nodeID) AS nFacts FROM tblMasterFact) u
WHERE tblMasterFact.nodeId = u.nodeID AND tblMasterFact.fact = u.fact
"""
)

In [48]:
conn.commit()
conn.close()

### QC of MasterFact output

In [24]:
pdConn = sq.connect('./SEC_reporting.sqlite')

In [ ]:
#check most frequent facts
pd.read_sql(
"""
SELECT fact, COUNT(DISTINCT ticker) 
FROM tblFact 
GROUP BY fact ORDER BY 2 DESC
"""
,pdConn)

In [ ]:
#total companies extracted
pd.read_sql("SELECT COUNT(DISTINCT ticker) FROM tblFact",pdConn)

,COUNT(DISTINCT ticker)
0,8773


In [ ]:
#total rows
pd.read_sql("SELECT COUNT(*) FROM tblFact",pdConn)

,COUNT(*)
0,21450664


In [ ]:
pd.read_sql("SELECT COUNT(*) FROM tblMasterFact",pdConn)

,COUNT(*)
0,9192878


In [ ]:
#check duplicates

pd.read_sql(
"""
SELECT ticker,fact,fy 
FROM tblMasterFact
GROUP BY ticker,fact,fy 
HAVING COUNT(DISTINCT val) > 1
"""
,pdConn)

#the selection of the following fields in the dense_rank in MasterFacts table was not sufficientto find unique combination
#end DESC, julianday(end)-julianday(start) DESC, filed DESC
#however this test show that no real ambuguity is left
#we can replace the dense_rak with row_number

,ticker,fact,fy


In [ ]:
#check duplicates

pd.read_sql(
"""
SELECT ticker,fact,fy 
FROM tblMasterFact
GROUP BY ticker,fact,fy 
HAVING COUNT(*) > 1
"""
,pdConn)

,ticker,fact,fy


In [ ]:
#check cases where fy is null
pd.read_sql(
"""
SELECT DISTINCT ticker
FROM tblMasterFact
WHERE fy IS NULL
"""
,pdConn)

,ticker


In [ ]:

pd.read_sql(
"""
SELECT A.fact, COUNT(DISTINCT ticker)
FROM tblMasterFact A
INNER JOIN tblFactList B
ON A.fact = B.fact
WHERE fy >= '2012'
GROUP BY A.fact
ORDER BY 2 DESC
"""
,pdConn)

,fact,COUNT(DISTINCT ticker)
0,Assets,8569
1,LiabilitiesAndStockholdersEquity,8363
2,NetIncomeLoss,8216
3,StockholdersEquity,8175
4,RetainedEarningsAccumulatedDeficit,8117
...,...,...
63,StandardProductWarrantyAccrual,290
64,OtherAccruedLiabilitiesNoncurrent,261
65,StandardProductWarrantyAccrualPayments,242
66,StandardProductWarrantyAccrualWarrantiesIssued,219


In [25]:
pdConn.close()

### Main analysis

#### Discre Approach (obsolete)

In [8]:
#Create Table of Bipartite graph with nodes (tickerFy) -[HAS_PROPERTY]-> (factDecile)
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE TABLE  tblBipartiteLinks AS
SELECT *
        ,COUNT(*) OVER (PARTITION BY tickerFy) AS Nbfacts
        ,DENSE_RANK() OVER (ORDER BY tickerFy ASC) AS RN
FROM 
    (
    SELECT 
      ticker
      ,val
      ,ticker || '_' || fy AS tickerFy
      ,fact || '_' || CAST((NTILE(10) OVER (PARTITION BY fact ORDER BY val ASC)) AS TEXT) AS factDecile
    FROM tblMasterFact
    ) X
"""    
)
conn.commit()
conn.close()

In [9]:
#Create Index on factDecile
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE INDEX fd ON tblBipartiteLinks(factDecile)
"""    
)
conn.commit()
conn.close()

In [31]:
#Projected graph table
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE TABLE tblProjectedWeightedGraph (SourceTickerFy TEXT, TargetTickerFy TEXT, JaccardSimilarity REAL)
"""    
)
conn.commit()
conn.close()

In [4]:
#Create list for the iteration 
conn = sq.connect('./SEC_reporting.sqlite')
tickerFyList = pd.read_sql(
"""
SELECT DISTINCT tickerFy
FROM tblBipartiteLinks A1
ORDER BY RN
"""
,conn).values
conn.close()

In [32]:
#Create projected graph
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()

for t in tqdm(tickerFyList):
    cursor.execute(
    """
    INSERT INTO tblProjectedWeightedGraph (SourceTickerFy,TargetTickerFy,JaccardSimilarity)
    
    SELECT 
             A1.tickerFy AS SourceTickerFy
            ,A2.tickerFy AS TargetTickerFy
            ,COUNT(*)*1./(A1.Nbfacts + A2.NbFacts - COUNT(*)) JaccardSimilarity
    FROM (
            SELECT *
            FROM tblBipartiteLinks
            WHERE tickerFy = '""" + t[0] + """'
         ) A1
    INNER JOIN tblBipartiteLinks A2 
        ON A1.factDecile = A2.factDecile
    WHERE A1.RN < A2.RN 
    GROUP BY A1.tickerFy,A2.tickerFy
    """    
    )
    
conn.commit()
conn.close()

100%|██████████| 57556/57556 [12:44:49<00:00,  1.25it/s]  


In [10]:
#Drop tables to save space - refer to backup sqlite SEC_reporting_20221119.sqlite
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
DROP TABLE tblBipartiteLinks
"""    
)
conn.commit()
conn.close()

In [11]:
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
DROP TABLE tblProjectedWeightedGraph
"""    
)
conn.commit()
conn.close()

In [12]:
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
VACUUM
"""    
)
conn.commit()
conn.close()

##### Exploration

In [11]:
pdConn = sq.connect('./SEC_reporting.sqlite')

In [ ]:
dtale.show(pd.read_sql(
"""
SELECT *
FROM tblProjectedWeightedGraph
WHERE SourceTickerFy LIKE 'AAPL%' OR TargetTickerFy LIKE 'AAPL%'
ORDER BY JaccardSimilarity DESC
LIMIT 200
"""
,pdConn))

In [ ]:
dtale.show(pd.read_sql(
"""
SELECT 
    A.tickerFy AS A_tickerFy,A.val AS A_val,
    B.tickerFy AS B_tickerFy,B.val AS B_val,B.factDecile AS B_factDecile
FROM (SELECT * FROM tblBipartiteLinks WHERE tickerFy = 'AMGN_2016')  A
INNER JOIN (SELECT * FROM tblBipartiteLinks WHERE tickerFy ='AAPL_2017')  B
ON A.factDecile = B.factDecile
ORDER BY A.tickerFy , B.tickerFy 
"""
,pdConn))

In [ ]:
pd.read_sql(
"""
SELECT MIN(JaccardSimilarity) JaccardSimilarity
FROM 
    (
    SELECT PERCENT_RANK() OVER (ORDER BY JaccardSimilarity ASC) RN,JaccardSimilarity
    FROM tblProjectedWeightedGraph
    ) X
WHERE RN = 0.95

"""
,pdConn)

In [59]:
pdConn.close()

In [ ]:
dtale.show(pd.read_sql(
"""
SELECT *
FROM tblProjectedWeightedGraph
WHERE SourceTickerFy LIKE 'BRK-A%' OR TargetTickerFy LIKE 'BRK-A%'
ORDER BY JaccardSimilarity DESC
LIMIT 200
"""
,pdConn))

In [48]:
pdConn = sq.connect('./SEC_reporting.sqlite')

In [33]:
pd.read_sql(
"""
SELECT COUNT(*)
FROM tblProjectedWeightedGraph
"""
,pdConn)

,COUNT(*)
0,1169530557


In [6]:
pd.read_sql(
"""
SELECT DISTINCT tickerFy
FROM tblBipartiteLinks A1
"""
,pdConn)

,tickerFy
0,AAAU_2018
1,AAAU_2019
2,AAAU_2020
3,AAAU_2021
4,AAC-UN_2021
...,...
57551,Z_2017
57552,Z_2018
57553,Z_2019
57554,Z_2020


In [30]:
pdConn.close()

#### Percentile Rank approach

In [3]:
#Create similarity matrix
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE TABLE tblSimilarityScoring  (SourceTickerFy VARCHAR, TargetTickerFy VARCHAR, SimilarityScore REAL)
"""    
)
conn.commit()
conn.close()

In [2]:
#Create similarity matrix
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
DROP TABLE tblSimilarityScoring
"""    
)
conn.commit()
conn.close()

In [5]:
#Create Index on fac
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE INDEX fct ON tblMasterFact(fact)
"""    
)
conn.commit()
conn.close()

In [11]:
#Create list for the iteration 
conn = sq.connect('./SEC_reporting.sqlite')
tickerFyList = pd.read_sql(
"""
SELECT DISTINCT nodeID
FROM tblMasterFact A1
ORDER BY nodeID
"""
,conn).values
conn.close()

In [ ]:
dtale.show(tickerFyList)

In [7]:
#Calculate similarity score
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()

for t in tqdm(tickerFyList):
    cursor.execute(
    """
    INSERT INTO tblSimilarityScoring (SourceTickerFy,TargetTickerFy,SimilarityScore)
    
    SELECT 
             A1.nodeID AS SourceTickerFy
            ,A2.nodeID AS TargetTickerFy
            ,(1.*COUNT(*)/(A1.numberOfFactsPerNode+A2.numberOfFactsPerNode-COUNT(*)))*(1./(1+AVG(ABS(A1.percentileVal-A2.percentileVal)))) SimilarityScore
    FROM (
            SELECT *
            FROM tblMasterFact
            WHERE nodeID = '""" + t[0] + """'
         ) A1
    INNER JOIN tblMasterFact A2 
        ON A1.fact = A2.fact
    WHERE A1.nodeID < A2.nodeID 
    GROUP BY A1.nodeID,A2.nodeID 
    """    
    )
conn.commit()
conn.close()

100%|██████████| 57556/57556 [36:47:04<00:00,  2.30s/it]   


In [ ]:
#Create truncated similarity scoring matrix with top 1% of score
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
CREATE TABLE tblSimilarityScoringTruncated (SourceTickerFy VARCHAR, TargetTickerFy VARCHAR, SimilarityScore REAL)
"""    
)
conn.commit()
conn.close()

In [ ]:
dtale.show(pd.read_sql(
"""
SELECT COUNT(*)
FROM tblSimilarityScoring
"""
,pdConn))
#1'632'658'223
print (1632658223*0.01)
#16326582

In [ ]:
#Create truncated similarity scoring matrix with top 1% of score
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
INSERT INTO tblSimilarityScoringTruncated (SourceTickerFy,TargetTickerFy,SimilarityScore)

SELECT SourceTickerFy,TargetTickerFy,SimilarityScore
FROM tblSimilarityScoring
ORDER BY SimilarityScore DESC
LIMIT 16326582
"""    
)
conn.commit()
conn.close()

In [4]:
#Drop similarity matrix to save space
conn = sq.connect('./SEC_reporting.sqlite')
cursor = conn.cursor()
cursor.execute(
"""
DROP TABLE tblSimilarityScoring
"""    
)
cursor.execute(
"""
VACUUM
"""    
)
conn.commit()
conn.close()

##### Exploration

In [3]:
pdConn = sq.connect('./SEC_reporting.sqlite')

In [43]:
#Test outcome on Silicon Valley Bank
output=pd.read_sql(
"""
SELECT*
FROM
    (
    SELECT TargetTickerFy, SimilarityScore
    FROM tblSimilarityScoringTruncated
    WHERE SourceTickerFy = 'SIVB_2021' 
        AND SUBSTRING(SourceTickerFy,1,INSTR(SourceTickerFy,'_')) <> SUBSTRING(TargetTickerFy,1,INSTR(TargetTickerFy,'_'))
    UNION
    SELECT SourceTickerFy, SimilarityScore
    FROM tblSimilarityScoringTruncated
    WHERE TargetTickerFy = 'SIVB_2021' 
        AND SUBSTRING(SourceTickerFy,1,INSTR(SourceTickerFy,'_')) <> SUBSTRING(TargetTickerFy,1,INSTR(TargetTickerFy,'_'))
    ) X

ORDER BY SimilarityScore DESC
LIMIT 500
"""
,pdConn)

In [40]:
pd.read_sql(
"""
SELECT ABS(A.percentileVal-B.percentileVal) ABS_error, A.fact,A.percentileVal Aval, B.percentileVal Bval,A.numberOfFactsPerNode AFacts, B.numberOfFactsPerNode Bfacts
FROM tblMasterFact A
INNER JOIN tblMasterFact B
ON A.fact = B.fact
WHERE A.nodeID = 'SIVB_2021' AND B.nodeID = 'HBANM_2021'
ORDER BY 1
"""
,pdConn)

In [34]:
pd.read_sql(
"""
SELECT COUNT(*)
FROM tblSimilarityScoringTruncated
"""
,pdConn)

,COUNT(*)
0,16326582


In [37]:
pd.read_sql(
"""
SELECT DISTINCT SourceTickerFy
FROM tblSimilarityScoringTruncated
"""
,pdConn)

,SourceTickerFy
0,AAC-UN_2021
1,AAC-WT_2021
2,AACIU_2021
3,AACIW_2021
4,AAIC-PB_2011
...,...
56856,MLYF_2021
56857,AFHIF_2021
56858,YCQH_2021
56859,ZTS_2021


In [12]:
output=pd.read_sql(
"""
SELECT DISTINCT TargetTickerFy
FROM tblSimilarityScoringTruncated
UNION
SELECT DISTINCT SourceTickerFy
FROM tblSimilarityScoringTruncated 
"""
,pdConn)